# Projet maison n° 3

In [1]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [2]:
import os
os.chdir ("/Users/Alex/JUPYTER LAB PC FIXE/names")

In [3]:
# names us
def df_names_us():
    
    df = pd.DataFrame(columns=['year','name', 'gender', 'births'])

    for i in range(1880,2021):
        df1 = pd.read_csv(('yob{}.txt').format(i), names=['name', 'gender', 'births'])
        df1['year'] = i 
        df = df.append(df1)
        df = df[['year','name', 'gender', 'births']]

    return df

In [4]:
df_test = df_names_us()
df_test

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
31266,2020,Zykell,M,5
31267,2020,Zylus,M,5
31268,2020,Zymari,M,5
31269,2020,Zyn,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [5]:
# names fr
def df_names_fr():
    
    # on importe le csv
    df = pd.read_csv('/Users/Alex/JUPYTER LAB PC FIXE/nat2020.csv',sep=';')
    
    # on renomme les colonnes
    df = df.rename(columns={'sexe':'gender','preusuel':'name','annais':'year','nombre':'births'})
    
    # order columns
    df = df[['year', 'name', 'gender', 'births']]
    
    # on change les types
    df['year']   = df['year'].astype("object")
    df['gender'] = df['gender'].astype("int")
    df['name']   = df['name'].astype("string")
    df['births'] = df['births'].astype("int")
    
    # on recode le genre
    df['gender'] = df['gender'].map({1:'M',2:'F'})
    
    # Seuls les prénoms de 2 caractères et plus sont conservés
    df = df[df['name'].str.len()>=2]
    
    # on met la 1ere lettre en majuscule
    df['name'] = df['name'].str.capitalize()
    
    # suppression lignes inutilisables
    df = df[df['year'] != 'XXXX']
    df = df[df['name'] != '_prenoms_rares']
    
    # On trie les données
    df = df.sort_values(
    by = ['year','gender','births','name'],
    ascending = [True,True,False,True])
    
    # reset index
    df = df.reset_index(drop=True)
    
    return df

In [6]:
df_test = df_names_fr()
df_test

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
...,...,...,...,...
630401,2020,Élouen,M,3
630402,2020,Énes,M,3
630403,2020,Éphraïm,M,3
630404,2020,Éros,M,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [26]:
# taux de change
def df_taux_change(devises):
    # import CSV
    df=pd.read_csv('/Users/Alex/JUPYTER LAB PC FIXE/Webstat_Export_20211006.csv',sep=';')
    df=df.iloc[5:]
    
    # rename Titre en date
    df=df.rename(columns={'Titre :':'Date'})
    
    # renommer colonnes en devise 
    df=df.rename(columns=dict(zip(list(df.columns[1:]), list(df.columns[1:].map(lambda x: x[-4:-1])))))
    
    # l'index doit correspondre aux dates
    df['Date']=pd.to_datetime(df['Date'],dayfirst=True)
    df=df.set_index('Date')
    
    # supprime les valeurs nulles
    df=df.replace(',','.',regex=True)
    df=df.apply(pd.to_numeric, errors='coerce')
    
    #supprimer les na 
    df=df.dropna()
    
    return df[devises]

In [27]:
df_taux_change(['EUR','USD','CHF'])

ParserError: Error tokenizing data. C error: Expected 20 fields in line 602, saw 39


### Tests

In [15]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [16]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [17]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... FAIL
test_df_names_us (__main__.Lesson4Tests) ... FAIL
test_df_taux_change (__main__.Lesson4Tests) ... ERROR

ERROR: test_df_taux_change (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-15-b976908de6f7>", line 31, in test_df_taux_change
    df = df_taux_change(['CHF', 'GBP', 'USD'])
  File "<ipython-input-13-b490ae0c8cc7>", line 4, in df_taux_change
    df=pd.read_csv('/Users/Alex/JUPYTER LAB PC FIXE/Webstat_Export_20211006.csv',sep=';')
  File "C:\Users\Alex\anaconda3\lib\site-packages\pandas\io\parsers.py", line 610, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "C:\Users\Alex\anaconda3\lib\site-packages\pandas\io\parsers.py", line 468, in _read
    return parser.read(nrows)
  File "C:\Users\Alex\anaconda3\lib\site-packages\pandas\io\parsers.py", line 1057, in read
    index, columns, col_dict = self._engine.read(nrows)
 